In [2]:
# import the required libraries
import numpy as np
import pandas as pd
import psycopg2 as pg2
from psycopg2 import sql

### Connect to the Azure PostGRE SQL server


In [3]:
# set the required names and passwords
host = "azureps1.postgres.database.azure.com"
dbname = "datavalidation"
user = "admin_azureps1"
password = "************"
sslmode = "require"

In [4]:
# Construct connection string

conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = pg2.connect(conn_string)
print("Connection established")

cursor = conn.cursor()


Connection established


In [8]:
#just to check whether connection is working or not
q1 = sql.SQL("select * from test1")
cursor.execute(q1)
rows = cursor.fetchall()
rows

[(2, 'testinput')]

### Import the dataset and create a dynamic table

In [12]:
df = pd.read_csv('customerKYC.csv')  #import the dataset
cols = list(df.columns.values)       # Extract the column names as a list
datatypes = list(df.dtypes)          ## Extract the column data types as a list

def createTable(table_name, allcolumns, alldatatypes):
    fields = []
    for i in range(len(allcolumns)):
        columnName = allcolumns[i]
        coldatatype = str(alldatatypes[i])

        if coldatatype == 'int64':
            coldatatype = 'int'
        elif coldatatype == 'float64':
            coldatatype = 'float'
        else:
            coldatatype = 'varchar' 
        

        fields.append( sql.SQL( "{} {}" ).format( sql.Identifier(columnName), sql.SQL(coldatatype) ) )

        #This field will contain all columns as (columnName, columnDatatype) tuple

    #Write the query to create the table
    qry = sql.SQL( "CREATE TABLE {tbl_name} ( {columnsAndDatatype} );" ).format(
        tbl_name = sql.Identifier( table_name ),
        columnsAndDatatype = sql.SQL( ', ' ).join( fields ))

    #Execute the query
    cursor.execute(qry)

    return 'Table has been created and columns are added successfuly'


createTable(table_name='kyc_data', allcolumns=cols,alldatatypes=datatypes)


'Table has been created and columns are added successfuly'

### Commit the changes

In [13]:
conn.commit()

### Close the cursor and close the connection

In [19]:
conn.close()
cursor.close()